#### TF描述卷积层
    tf.keras.layers.Conv2D (
    filters = 卷积核个数,
    kernel_size = 卷积核尺寸, #正方形写核长整数，或（核高h，核宽w）
    strides = 滑动步长, #横纵向相同写步长整数，或使用元组形式给定(纵向步长h，横向步长w)，默认1
    padding = “same” or “valid”, #使用全零填充是“same”，不使用是“valid”（默认）
    activation = “ relu ” or “ sigmoid ” or “ tanh ” or “ softmax”等 , #给出激活函数如有BN（批标准化操作）此处不写
    input_shape = (高, 宽 , 通道数) #输入特征图维度，可省略
)

    eg：
    model = tf.keras.models.Sequential([
    Conv2D(6, 5, padding='valid', activation='sigmoid'),#第一层：6个卷积核，卷积核尺寸5，步长1，不使用全零填充，sigmoid激活函数
    MaxPool2D(2, 2),
    Conv2D(6, (5, 5), padding='valid', activation='sigmoid'),#第二层
    MaxPool2D(2, (2, 2)),
    Conv2D(filters=6, kernel_size=(5, 5),padding='valid', activation='sigmoid'),#第三层
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(10, activation='softmax')
])

#### 批标准化（Batch Normalization， BN）
    标准化：使数据符合0均值，1为标准差的分布。
    批标准化：对一小批数据（batch），做标准化处理 ，使数据回归标准正态分布，常用在卷积操作和激活操作之间。
    
$$ H_i^{/k}\frac{H_i^k-\mu _{batch}^k}{\sigma _{batch}^k} $$
$$ H_i^k:批标准化前，第k个卷积核，输出特征图中第 i 个像素点$$
$$ \mu _{batch}^k:批标准化前，第k个卷积核，batch张输出特征图中所有像素点平均值$$
$$\sigma _{batch}^k:批标准化前，第k个卷积核，batch张输出特征图中所有像素点标准差 $$

    BN操作：
    将原本偏移的数据，重新拉回到0均值，
    使进入激活函数的数据分布在激活函数线性区，
    使得输入数据的微小变化，更明显的体现到激活函数的输出，
    提升了激活函数对数据的区分力。
    
    但是这种简单的特征数据标准化，使特征数据完全满足标准正态分布，
    集中在激活函数中心的线性区域，使使激活函数丧失了非线性特性，
    因此，在BN操作中为每个卷积核引入了两个可训练参数：
    缩放因子γ；偏移因子β； 
$$ X_i^k = \gamma _k H_i^{/k} + \beta _k$$
    在反向传播时，缩放因子γ、偏移因子β会与其他代训练参数一起被训练优化，
    使标准正态分布后的特征数据，通过缩放因子和偏移因子，优化了特征数据分布的宽窄和偏移量，
    保证了网络的非线性表达力
    
    结构：
    卷积Convolutional
    批标准化BN
    激活Activation
    
    code：
    TF描述批标准化：tf.keras.layers.BatchNormalization()
    eg：
    model = tf.keras.models.Sequential([
    Conv2D(filters=6, kernel_size=(5, 5), padding='same'), # 卷积层
    BatchNormalization(), # BN层
    Activation('relu'), # 激活层
    MaxPool2D(pool_size=(2, 2), strides=2, padding='same'), # 池化层
    Dropout(0.2), # dropout层
    ])
    

#### 池化Pooling
    池化用于减少特征数据量。
    最大值池化可提取图片纹理，均值池化可保留背景特征。
    max pool：输出池化框中像素点的最大值
    mean pool：输出池化框中像素点的均值
    
    code：
    TF描述池化
    
    tf.keras.layers.MaxPool2D(#最大池化-纹理
    pool_size=池化核尺寸，#正方形写核长整数，或（核高h，核宽w）
    strides=池化步长，#步长整数， 或(纵向步长h，横向步长w)，默认为pool_size
    padding=‘valid’or‘same’ #使用全零填充是“same”，不使用是“valid”（默认）
    )
    
    tf.keras.layers.AveragePooling2D(#均值池化-特征
    pool_size=池化核尺寸，#正方形写核长整数，或（核高h，核宽w）
    strides=池化步长，#步长整数， 或(纵向步长h，横向步长w)，默认为pool_size
    padding=‘valid’or‘same’ #使用全零填充是“same”，不使用是“valid”（默认）
    )
    
    eg：
    model = tf.keras.models.Sequential([
    Conv2D(filters=6, kernel_size=(5, 5), padding='same'), # 卷积层
    BatchNormalization(), # BN层
    Activation('relu'), # 激活层
    MaxPool2D(pool_size=(2, 2), strides=2, padding='same'), # 池化层：池化核2*2，池化步长2，使用全零填充
    Dropout(0.2), # dropout层
    ])

#### 舍弃Dropout
    在神经网络训练时，将一部分神经元按照一定概率从神经网络中暂时舍弃。神经网络使用时，被舍弃的神经元恢复链接。
    
    code：
    TF描述舍弃
    tf.keras.layers.Dropout(舍弃的概率)

    model = tf.keras.models.Sequential([
    Conv2D(filters=6, kernel_size=(5, 5), padding='same'), # 卷积层
    BatchNormalization(), # BN层
    Activation('relu'), # 激活层
    MaxPool2D(pool_size=(2, 2), strides=2, padding='same'), # 池化层
    Dropout(0.2), # dropout层：随机舍弃20%的神经元
    ])

#### 卷积神经网络：借助卷积核提取特征后，送入全连接网络
    
    卷积神经网络网络的主要模块：
    
    卷积Convolutional
    批标准化BN
    激活Activation
    池化Pooling
    
    全连接FC
    
    卷积就是特征提取器，就是‘CBAPD’
    model = tf.keras.models.Sequential([
    
    C    Conv2D(filters=6, kernel_size=(5, 5), padding='same'), # 卷积层
    B    BatchNormalization(), # BN层
    A    Activation('relu'), # 激活层
    P    MaxPool2D(pool_size=(2, 2), strides=2, padding='same'), # 池化层
    D    Dropout(0.2), # dropout层
    
    ])